<a href="https://colab.research.google.com/github/cdario79/AffidabilitaCreditiziaCC/blob/main/Affidabilita_Creditizia_CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Previsione dell'affidabilità creditizia per il rilascio della carta di credito

## 🧩 1. Esplorazione e comprensione del problema
**🎯 Riferimento testo:**  
> “il tuo primo incarico consiste nel realizzare un modello in grado di stimare l'affidabilità creditizia dei clienti, al fine di aiutare il team dedicato a comprendere se accettare o meno la richiesta per il rilascio della carta di credito.”

**✔ Attività:**
- Comprendere il significato del **target** (TARGET = 1 → affidabile, 0 → non affidabile).
- Analizzare il contesto decisionale.
- Definire metriche di valutazione adeguate (Accuracy, Precision, Recall, AUC-ROC, ecc.).

---

## 📥 2. Caricamento e ispezione iniziale del dataset
**📄 Riferimento testo:**  
> “I dati sono disponibili in un file CSV [...] credit_scoring.csv”  
> “Colonne del dataset: ID, CODE_GENDER, FLAG_OWN_CAR, [...] TARGET”

**✔ Attività:**
- Caricare il file `credit_scoring.csv`.
- Esplorare struttura, valori nulli, tipi di dato.
- Verificare la distribuzione della variabile TARGET.

---

## 🧹 3. Pulizia e preparazione dei dati
**🧾 Riferimento testo:**  
> “Il file credit_scoring.csv contiene le informazioni dei correntisti che hanno richiesto l’apertura di una linea di credito.”

**✔ Attività:**
- Gestione dei valori nulli (es. OCCUPATION_TYPE).
- Trasformazione variabili categoriche (one-hot encoding).
- Elaborazione delle colonne temporali (`DAYS_BIRTH`, `DAYS_EMPLOYED`).
- Rimozione colonne non informative (es. ID).

---

## 📊 4. Analisi esplorativa dei dati (EDA)
**📌 Riferimento testo:**  
> “Ti vengono consegnati i dati anonimizzati di clienti che hanno già ottenuto la carta di credito e ne pagano regolarmente le rate.”

**✔ Attività:**
- Analizzare correlazioni tra feature e TARGET.
- Visualizzare distribuzioni e valori anomali.
- Individuare pattern e potenziali variabili rilevanti.

---

## 🤖 5. Costruzione e valutazione del modello predittivo
**📈 Riferimento testo:**  
> “Devi realizzare un modello che preveda il target dato, ovvero la variabile TARGET che indica se il cliente ha una buona affidabilità creditizia.”

**✔ Attività:**
- Separare train/test set.
- Provare più modelli (Logistic Regression, Random Forest, XGBoost).
- Valutare le prestazioni con metriche adeguate.
- Gestire class imbalance (SMOTE o pesi).

---

## 🧠 6. Interpretabilità del modello (Explainability)
**⭐ Riferimento testo:**  
> “Punto bonus: il team deve essere in grado di fornire una motivazione. Questo significa che il tuo modello deve fornire delle indicazioni facilmente interpretabili.”

**✔ Attività:**
- Calcolare l’importanza delle feature.
- Utilizzare tecniche come SHAP o LIME.
- Generare spiegazioni individuali per le predizioni.

---

## 🧪 7. Validazione e ottimizzazione
**🔍 Riferimento testo:**  
> Implicito nella richiesta di "realizzare un modello" affidabile.

**✔ Attività:**
- Validazione incrociata (cross-validation).
- Ottimizzazione iperparametri (Grid Search/Random Search).
- Prevenzione dell’overfitting.

---

## 🗂 8. Consegna, documentazione e comunicazione
**📤 Riferimento testo:**  
> “aiutare il team dedicato a comprendere se accettare o meno la richiesta per il rilascio della carta di credito.”

**✔ Attività:**
- Documentazione tecnica del modello.
- Presentazione dei risultati al team (tecnica e business-friendly).
- Output del modello salvato e pronto per l’integrazione (es. API, notebook, script).

## 📥 1. Caricamento Librerie e Dati

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

import shap

# Per visualizzare meglio i grafici
sns.set(style="whitegrid")

In [2]:
# Caricamento dataset
df = pd.read_csv("https://proai-datasets.s3.eu-west-3.amazonaws.com/credit_scoring.csv")

# Visualizza prime righe
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TARGET
0,5008804,M,Y,Y,0,424380.57,Working,Higher education,Civil marriage,Rented apartment,-11778.0,-4438.0,1.0,1.0,0.0,0.0,NaN,2.0,0
1,5008805,M,Y,Y,0,421593.52,Working,Higher education,Civil marriage,Rented apartment,-11777.0,-4459.0,1.0,1.0,0.0,0.0,NaN,2.0,0
2,5008806,M,Y,Y,0,110958.51,Working,Secondary / secondary special,Married,House / apartment,-21471.0,-1144.0,1.0,0.0,0.0,0.0,Security staff,2.0,0
3,5008808,F,N,Y,0,267497.96,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-18489.0,-3122.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1
4,5008809,F,N,Y,0,263419.09,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19093.0,-3070.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1


🧼 2. Preprocessing Base

In [ ]:
# Controlla valori nulli
df.isnull().sum()

In [ ]:
# Riempimento base dei valori mancanti
df.fillna("Missing", inplace=True)

In [ ]:
# Encoding delle variabili categoriche
categorical_cols = df.select_dtypes(include="object").columns

le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

🔀 3. Divisione Train/Test

In [ ]:
X = df.drop(columns=["TARGET", "ID"])
y = df["TARGET"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

🌲 4. Addestramento Modello (Random Forest)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

🔍 5. Interpretazione con SHAP

In [ ]:
# Inizializza SHAP
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# Grafico feature importance
shap.plots.bar(shap_values)

✅ Prossimi Passi (suggeriti)
	•	Migliorare la gestione dei missing values
	•	Provare altri modelli (XGBoost, LightGBM, Logistic Regression…)
	•	Bilanciare la classe se sbilanciata (class_weight='balanced' o SMOTE)
	•	Ottimizzazione iperparametri (GridSearchCV o RandomizedSearchCV)
	•	Interfaccia utente per spiegazioni su richiesta